# Панельная регрессия с Linearmodels. Подгонка регрессии через матрицы дизайна

In [ ]:
import numpy as np
import pandas as pd

from linearmodels import PooledOLS     # Pool модель
from linearmodels import RandomEffects # RE-модель
from linearmodels import PanelOLS      # FE-модель

from linearmodels.panel import compare # несколько регрессий в одной таблице

from statsmodels.api import add_constant # добавление столбца из единиц

## Загрузка и подготовка датасета

In [ ]:
df = pd.read_csv('Wages.csv')

# зададим панельную структуру через мульти-индекс
# id - индивидуальный индекс, time - временной индекс
panel_df = df.set_index(['id', 'time'])
panel_df

## Панельные регрессии
Рассмотрим модель `lwage` на `ed, exp, exp**2, south, smsa, ind`  в трёх  вариантах:
- Pool
- Random effects
- Fixed effects

Т.к. набор переменных один и тот же, матрицы регрессионного дизайна одни и те же.

Для создания матриц дизайна нужно учесть:

1. квадрат `exp**2` нужно будет добавить вручную
2. переменные `south, smsa` категориальные. их нужно преобразовать в dummy такеж вручную

**Замечание** подгоним модели с настройками по умолчанию, т.е. с неробастной оценкой ковариационной матрицы

In [ ]:
# преобразование категориальных в dummy
panel_df = pd.get_dummies(data=panel_df, columns=['south','smsa','married', 'sex', 'union', 'black'], dtype=int)
panel_df

In [ ]:
#  Матрицы дизайна
y = panel_df['lwage']

X = add_constant(panel_df[['ed', 'exp', 'south_yes', 'smsa_yes', 'ind']])
# добавим квадрат exp**2 после exp (после столбца с индексом 2), т.е. как столбец с индексом loc=3 и именем 'exp**2'
X.insert(loc=3, column='exp**2', value=panel_df['exp']**2)
X

### Pool-модель

In [ ]:
mod_pl = PooledOLS(dependent=y, exog=X)
res_pl = mod_pl.fit()

res_pl.summary

### RE-модель

In [ ]:
mod_re = RandomEffects(dependent=y, exog=X)
res_re = mod_re.fit()

res_re.summary

## FE-модель
**Важно**:
- указать какие эффекты включаем: `entity_effects` и `time_effects` (по умолчанию оба `False`)
- указать `drop_absorbed=True` для контроля постоянных во времени переменных

In [ ]:
mod_fe = PanelOLS(dependent=y, exog=X, entity_effects=True, drop_absorbed=True)
res_fe = mod_fe.fit()

res_fe.summary

### Три модели в одной таблице
Модели оформляем в виде словаря

In [ ]:
compare(results={'Pool':res_pl, 'RE':res_re, 'FE':res_fe}, precision='std-errors', stars=True)